В этом ноутбуке кортежи мнений предсказываются с помощью модели gpt-4o или любой другой модели, доступной через сервис openrouter.

Здесь используется OpenAI API, также необходим Openrouter API key.

Рекомендуется запускать из Google Colab, а не локально, во избежание геополитических ограничений.

Для формирования затравок используются предсказания других больших языковых моделей (см. ноутбуки gpt4 и mistral в этом репозитории). Они берутся из .csv файлов, которых нет в открытом доступе.

Без этих файлов воспроизвести происходящие в этом ноутбуке не получится.

### Проверка (OpenAI API) ###

In [ ]:
import requests
import ast
import json
import os
import itertools
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
import random
from random import choices
import time

from openai import OpenAI

OPENROUTER_API_KEY = "YOUR_API_KEY"

# model = "anthropic/claude-3.5-sonnet"
model = "thedrummer/unslopnemo-12b:free" # бесплатная модель
# model = "openai/gpt-4o-2024-08-06"
url = "https://openrouter.ai/api/v1"

SEED = 42
random.seed(SEED)

client = OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=url,
)

params = {'temperature': 1.7,
         'top_p': 0.9,
         'max_tokens': 512,
         'seed': SEED}

completion = client.chat.completions.create(
    model=model,
    **params,
    messages=[
        {"role": "system", "content": "Ответь на вопрос."},
        {"role": "user", "content": "Тебя сделал Илон Маск?"},
    ],
)

print(completion.choices[0].message.content)

Нет, меня قامbling шесть assays Spotify Schlaf**,{Wprints Circlepol8/files  choose }.


### Подготовка данных ###

In [ ]:
import requests, zipfile, io

# download source files
url = 'https://raw.githubusercontent.com/rossyaykin/RuOpinionNE/refs/heads/main/src/src.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('')

In [ ]:
from src.utils import load_jsonl, save_jsonl, str2list, dict2tuple, extract_tuple, df2structure

In [ ]:
train_path = "full.jsonl"
test_path = "validation.jsonl"

url = 'https://raw.githubusercontent.com/rossyaykin/RuOpinionNE/refs/heads/main/data/full.jsonl'
train = load_jsonl(url, train_path)
url = 'https://raw.githubusercontent.com/rossyaykin/RuOpinionNE/refs/heads/main/data/validation.jsonl'
test = load_jsonl(url, test_path)

print(len(train), len(test))

2556 1316


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# здесь используются .csv файлы с предсказаниями моделей, которых на данный момент не в открытом доступе

mistral = pd.read_csv('/content/gdrive/My Drive/Mistral_bl_15shot_0.1temp.csv')
grok = pd.read_csv('/content/gdrive/My Drive/Grok_bl_15shot_0.1temp.csv')
claude = pd.read_csv('/content/gdrive/My Drive/Claude_bl_15shot_0.1temp.csv')
aug = [mistral, grok, claude]
# gold = parse_data('data/gold.jsonl')

In [ ]:
mistral.head()

,sent_id,text,target,pred
0,0,В числе участников президентской борьбы есть о...,[],[]
1,1,"Кандидатке на пост президента 54 года, она род...",[],[]
2,2,"Сама женщина заявила, что встречаться с сыном ...",[],[]
3,3,"Они снимали нас все эти 5 минут, что длилось с...",[],"[['NULL', 'Влад', 'ничего лишнего мне не сказа..."
4,4,"Кроме того, по словам женщины на щеке сына Све...",[],"[['NULL', 'сына', 'синяк', 'NEG']]"


In [ ]:
mistral.loc[3]['pred']

"[['NULL', 'Влад', 'ничего лишнего мне не сказал', 'NEG']]"

### Определения ###

In [ ]:
def aug_prompt(examples, variants, text):
    shots = '\n'.join([f'Текст: {pair[0]}\nОтвет: {pair[1]}' for pair in examples])
    variants = '\n'.join([str(x) for x in variants])
    return f"""Ты эксперт в оценке тональности.
Тебе нужно найти все негативные и позитивные отношения между сущностями в тексте и вывести их в следующем формате:
[источник отношения, объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если источником отношения является автор, то пиши:
['AUTHOR', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если выраженного источника нет, то пиши:
['NULL', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Допустимо вернуть пустой ответ:
[]
Не нужно давать пояснений к ответу.
Примеры
{shots}
Текст, котоырй нужно проанализировать:
{text}
Ответы экспертов к этому тексту:
{variants}
Ты можешь выбрать из этих ответов или ответить по-своему.
Твой ответ:"""

class Runner():
    def __init__(self, model, client, params, train, test, aug, n_shots = 12, sleeptime = 2):
        self.model = model
        self.client = client
        self.params = params
        self.train = train
        self.test = test
        self.n_shots = n_shots
        self.sleeptime = sleeptime
        self.aug = aug

    def run(self):
        results = list()
        for i in tqdm(range(len(self.test))):
            entry = self.test[i]
            time.sleep(self.sleeptime)
            examples = [dict2tuple(x) for x in choices(self.train, k = n_shots)]
            variants = [x.loc[i]['pred'] for x in self.aug]
            prompt = aug_prompt(examples, variants, entry['text'])
            completion = client.chat.completions.create(model=self.model,
                                                        **self.params,
                                                        messages=[{"role": "user", "content": prompt}],)
            result = completion.choices[0].message.content
            try:
                result = ast.literal_eval(result)
            except (SyntaxError, ValueError):
                print(f'bad response, iteration:{len(results)}')
                result = []
            results.append((entry['sent_id'],
                            entry['text'],
                            dict2tuple(entry)[1], # gold opinions
                            result)) # pred opinions
        return results

def get_path(temp, n_shots):
    path = f'./content/GPTaug/GPTaug_bl_{n_shots}shot_{temp}temp'
    # returns full path but without ".csv"
    return path

def save(dataframe, path, raw = True):
    outdir, outname = '/'.join(path.split('/')[:-1]), path.split('/')[-1]
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    if raw:
        dataframe.to_csv(f'{path}_raw.csv', index = False)
    else:
        dataframe.to_csv(f'{path}.csv', index = False)

### Тест на одном примере ###

In [ ]:
n_shots = 7
examples = [dict2tuple(x) for x in train[:n_shots]]
text, target = dict2tuple(test[n_shots])
variants = [x.loc[n_shots]['pred'] for x in aug]

sample_prompt = aug_prompt(examples, variants, text)
print(f'--Промпт--:\n\n{sample_prompt}')

--Промпт--:

Ты эксперт в оценке тональности.
Тебе нужно найти все негативные и позитивные отношения между сущностями в тексте и вывести их в следующем формате:
[источник отношения, объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если источником отношения является автор, то пиши:
['AUTHOR', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если выраженного источника нет, то пиши:
['NULL', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Допустимо вернуть пустой ответ:
[]
Не нужно давать пояснений к ответу.
Примеры
Текст: Президент Башкирии Муртаза Рахимов в очередной раз решил поменять главу своей администрации.
Ответ: [['Муртаза Рахимов', 'главу своей администрации', 'поменять', 'NEG']]
Текст: Вчера он уволил Азамата Сагитова, который возглавил башкирскую администрацию год назад после вынужденной отставки Радия Хабирова, сейчас занимающего пост заместителя начальника управления президента РФ по внутренней полит

In [ ]:
model = "openai/gpt-4o-2024-08-06"
# model = "anthropic/claude-3.5-sonnet"
# model = "thedrummer/unslopnemo-12b:free"

params = {'temperature': 0.1,
          'seed': SEED,
          'top_p': 0.9,
          'max_tokens': 512}

completion = client.chat.completions.create(
    model=model,
    **params,
    messages=[{"role": "user",
               "content": sample_prompt}],
)

response = completion.choices[0].message.content
result = ast.literal_eval(response)

print(f'--Текст--:\n{text}')
print(f'--Таргет--:\n{target}')
print(f'--Предикт--:\n{extract_tuple(result)}')

--Текст--:
Напомним, на казахстанском пограничном временном посту "Арканкерген" были обнаружены обгоревшие тела 14 пограничников и 1 егеря.
--Таргет--:
[]
--Предикт--:
[['NULL', '14 пограничников и 1 егеря', 'обгоревшие тела', 'NEG']]


### Инференс ###

In [ ]:
%%time
params = {'temperature': 0.1,
          'seed': SEED,
          'top_p': 0.9,
          'max_tokens': 512}
n_shots = 12
runner = Runner(model, client, params, train, test, aug, n_shots)

path = get_path(params['temperature'], n_shots)
result = runner.run()

100%|██████████| 1316/1316 [1:14:16<00:00,  3.39s/it]

CPU times: user 37.9 s, sys: 4.29 s, total: 42.2 s
Wall time: 1h 14min 16s


In [ ]:
print(path)
print(result[7])

./content/GPTaug/GPTaug_bl_12shot_0.1temp
(7, 'Напомним, на казахстанском пограничном временном посту "Арканкерген" были обнаружены обгоревшие тела 14 пограничников и 1 егеря.', [], [['NULL', '14 пограничников и 1 егеря', 'обгоревшие тела', 'NEG']])


### Результаты ###

In [ ]:
output = pd.DataFrame(result, columns = ['sent_id', 'text', 'target', 'pred'])
output.head()

,sent_id,text,target,pred
0,0,В числе участников президентской борьбы есть о...,[],[]
1,1,"Кандидатке на пост президента 54 года, она род...",[],[]
2,2,"Сама женщина заявила, что встречаться с сыном ...",[],[]
3,3,"Они снимали нас все эти 5 минут, что длилось с...",[],"[[AUTHOR, они, снимали нас все эти 5 минут, чт..."
4,4,"Кроме того, по словам женщины на щеке сына Све...",[],"[[Светлана, сына, синяк, NEG]]"


In [ ]:
# print(path)
# save(output, path)
output.to_csv(f'result_raw.csv', index = False)

In [ ]:
output = pd.DataFrame([(x[0], x[1], x[2], str2list(extract_tuple(x[3]))) for x in result],
                      columns = ['sent_id', 'text', 'target', 'pred'])
output.head()

,sent_id,text,target,pred
0,0,В числе участников президентской борьбы есть о...,[],[]
1,1,"Кандидатке на пост президента 54 года, она род...",[],[]
2,2,"Сама женщина заявила, что встречаться с сыном ...",[],[]
3,3,"Они снимали нас все эти 5 минут, что длилось с...",[],"[[AUTHOR, они, снимали нас все эти 5 минут, чт..."
4,4,"Кроме того, по словам женщины на щеке сына Све...",[],"[[Светлана, сына, синяк, NEG]]"


In [ ]:
# print(path)
# save(output, path, raw = False)
output.to_csv(f'result.csv', index = False)

In [ ]:
final = df2structure(output)
save_jsonl(final, 'result')

In [ ]:
final[40:50]

[{'sent_id': 40,
  'text': 'Телеканал отмечает, что США осуществят атаку, если «сочтут это необходимым».',
  'opinions': [{'Source': [['NULL'], ['0:0']],
    'Target': [['США'], ['24:27']],
    'Polar_expression': [['осуществят атаку'], ['28:44']],
    'Polarity': 'NEG'}]},
 {'sent_id': 41,
  'text': 'При этом он пояснил, что администрация не будет обсуждать потенциальные ответы, некоторые из них в России увидят, некоторые\xa0— нет.',
  'opinions': []},
 {'sent_id': 42,
  'text': 'ИИ-систему Google обучили на шести телешоу (таких как Newsnight, BBC Breakfast и Question Time), которые выходили в эфир с 2010 по 2015 года, «скормив» ей видео общей продолжительностью 5 тысяч часов (около 118 тысяч предложений).',
  'opinions': [{'Source': [['Google'], ['11:17']],
    'Target': [['ИИ-систему'], ['0:10']],
    'Polar_expression': [['обучили'], ['18:25']],
    'Polarity': 'POS'}]},
 {'sent_id': 43,
  'text': 'В Центральной Италии в августе 2016 года произошло сильное землетрясение.',
  'opini